In [4]:
import cv2
import os,numpy as np, cv2 , tensorflow as tf,time
import face_recognition

# encoding the known faces

In [5]:
known_faces_dir = "static/data/known/"
def get_encoded_faces(known_faces_dir):
    encoded = {}
    for name in os.listdir(f"{known_faces_dir}"):
        for filename in os.listdir(f"{known_faces_dir}/{name}"):
            face = face_recognition.load_image_file(f"{known_faces_dir}/{name}/{filename}")
            face_encodings = face_recognition.face_encodings(face)[0]
            encoded[name.split(".")[0]] = face_encodings
    return encoded;

# => detect the face on images 
# - draw the bounding box around the face

In [9]:
def detectOnImage(imagePath):
#     read the image for detecting
    image = cv2.imread(f"{imagePath}",1)
#   recognition on known faces
    faces = get_encoded_faces(known_faces_dir)
    faces_encoded = list(faces.values())
    known_face_names = list(faces.keys())
    colorsList = np.random.uniform(low=0,high=255, size = (len(known_face_names),3))
#     fetch face features
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image,face_locations)

    face_names = []
    for face_encoding in face_encodings:
#         compare the known faces and the new image
        matches = face_recognition.compare_faces(faces_encoded,face_encoding)
        name = 'unknown'
        color=(0,255,0);
#         count the distance between the known faces and unknown face(new image)
        faces_distances  = face_recognition.face_distance(faces_encoded,face_encoding) 
        best_match_index = np.argmin(faces_distances)
#     define the name of face if is a known face with a random color
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            color = colorsList[best_match_index]
        face_names.append(name)
        #     Draw the Bounding Box around the face with label
        for (top,right,bottom,left), name in zip(face_locations,face_names):            
            cv2.rectangle(image,(left-20,top-20),(right+20,bottom+20),color,2)
            cv2.putText(image, str(name), (left-20, top-25), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            cv2.imwrite('./static/images/',image)
#   display the final image after drawing the bounding box
#     cv2.imshow("result:",image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()  

# test on new image

In [10]:
img1 = "data/test/kr.jpg"
img2 = "data/test/img2.jpg"
detectOnImage(img1) 

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: array, upsample_num_times: int=0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x000001E49FBC4470>, None, 1

# detection the faces on video

In [4]:
def DetectOnVideo():
#     0 : for using the camera of my loptop
#     you can replace 0 with  path of your video if you wanna use a specific video !!!
        cap=cv2.VideoCapture(0)
        if (cap.isOpened() == False):
            print("Error in opning the video ....")
            return
        (success, image)=cap.read()    
        while success:
            faces = get_encoded_faces()
            faces_encoded = list(faces.values())
            known_face_names = list(faces.keys())
            
            face_locations = face_recognition.face_locations(image)
            face_encodings = face_recognition.face_encodings(image,face_locations)
            face_names=[]
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(faces_encoded, face_encoding)
                name='unknown'
                color=(255,0,0)
                face_distances=face_recognition.face_distance(faces_encoded,face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                face_names.append(name)
                for (top,right,bottom,left),name in zip(face_locations, face_names):
                    cv2.rectangle(image,(left-20,top-20),(right+20,bottom+20),color,2)
                    cv2.putText(image,str(name),(left-20,top-25),cv2.FONT_HERSHEY_PLAIN,2,color,2)
        
            cv2.imshow("result",image)
            key= cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            (success,image)=cap.read()
        cv2.destroyAllWindows()

In [138]:
DetectOnVideo()

# run this app on web page using flask

In [ ]:
from flask import Flask, render_template, request

In [ ]:
app = Flask(__name__)
@app.route('/',methods=['GET'])
def index():
    return render_template('index.html')
@app.route('/',methods=['POST'])
def FaceRecognition():
        
        img=request.files['image'];
        image_path='./static/data/known/match/' + img.filename;
        img.save(image_path);
        
        known_faces_dir='./static/data/known'
        return render_template('index.html');
@app.route('/predict',methods=['POST'])
def predictonImage():
    
        img=request.files['image2'];
        image_path='./static/images/' + img.filename;
        img.save(image_path);
        imageFinal = detectOnImage(image_path)
        imageFinal.save('./static/images/'+imageFinal.filename)
        return render_template('index.html',imgs=imageFinal);
if __name__ == '__main__':
    app.run()